### Importing libraries 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
import ast

### Loading the dataset

In [2]:
expenditures_df = pd.read_csv('../SavedData/dataset2_expenditures.csv')
payments_df= pd.read_csv('../SavedData/dataset2_payments.csv')
expenditures_users_df = pd.read_csv('../SavedData/dataset2_expenditures_by_users.csv')
payments_users_df = pd.read_csv('../SavedData/dataset2_payments_by_users.csv')
simulated_cleaned_tx_df = pd.read_csv('../SavedData/dataset2_cleaned.csv')

In [3]:
simulated_cleaned_tx_df

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name,Datetime
0,01/01/2023,00:00:00,678330503.0,2971.000000,1584.00,NaN,Westport Care Home,2023-01-01 00:00:00
1,01/01/2023,00:00:00,472213568.0,3792.000000,1950.00,NaN,Barbiee Boutique,2023-01-01 00:00:00
2,01/01/2023,00:00:00,472213568.0,3012.000000,-780.00,283027736.0,NaN,2023-01-01 00:00:00
3,01/01/2023,00:00:00,283027736.0,1787.000000,780.00,472213568.0,NaN,2023-01-01 00:00:00
4,01/01/2023,00:00:00,624500124.0,3226.000000,1825.00,NaN,Fat Face,2023-01-01 00:00:00
...,...,...,...,...,...,...,...,...
229189,06/12/2023,20:54:00,581655972.0,45935.206861,-41.06,NaN,Tesco,2023-12-06 20:54:00
229190,06/12/2023,20:55:00,786141370.0,-244.837500,-62.35,NaN,Sainsbury Local,2023-12-06 20:55:00
229191,06/12/2023,21:05:00,824916823.0,9709.172159,-32.94,NaN,Deliveroo,2023-12-06 21:05:00
229192,06/12/2023,21:13:00,366550080.0,26834.165794,-19.25,NaN,Amazon,2023-12-06 21:13:00


### Splitting the data into training and testing 

In [4]:
copied_simulated_tx_df = simulated_cleaned_tx_df.copy()

# Splitting the copied dataset into historical_transactions (95%) and testing_transactions (5%)
training_transactions, testing_transactions = train_test_split(copied_simulated_tx_df, test_size=0.1, random_state=42)

# Displaying the shapes to verify the split
training_transactions.shape, testing_transactions.shape


((206274, 8), (22920, 8))

### Splitting the training transactions into payments and expenditures

In [5]:
payments_training_df = training_transactions[training_transactions['Amount'] > 0]
expenditures_training_df = training_transactions[training_transactions['Amount'] < 0]

In [6]:
# Taking the absolute values of the amount column
expenditures_training_df["Amount"] = expenditures_training_df["Amount"].abs()

expenditures_training_df

/var/folders/rw/215myrns4md_n1k8nkyww6mw0000gn/T/ipykernel_32303/721076894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  expenditures_training_df["Amount"] = expenditures_training_df["Amount"].abs()


,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name,Datetime
108236,21/06/2023,09:03:00,809499628.0,10775.476914,48.10,NaN,Tesco,2023-06-21 09:03:00
204684,02/11/2023,19:22:00,518474507.0,15455.325500,29.19,NaN,Deliveroo,2023-11-02 19:22:00
19823,30/01/2023,11:29:00,655368335.0,2230.145000,99.98,NaN,Gamestation,2023-01-30 11:29:00
151016,25/08/2023,11:09:00,678851253.0,4313.628611,63.70,NaN,Fat Face,2023-08-25 11:09:00
26217,31/01/2023,23:59:00,163470640.0,60.014100,12.99,NaN,SquareOnix,2023-01-31 23:59:00
...,...,...,...,...,...,...,...,...
119879,03/07/2023,18:54:00,424397946.0,2450.930784,119.62,NaN,Sainsbury,2023-07-03 18:54:00
103694,11/06/2023,10:27:00,241459011.0,995.080013,15.98,NaN,Sports Direct,2023-06-11 10:27:00
131932,29/07/2023,14:02:00,744211756.0,-3127.708250,44.72,NaN,Tesco,2023-07-29 14:02:00
146867,16/08/2023,17:57:00,245690309.0,7640.773332,46.45,NaN,Tesco,2023-08-16 17:57:00


### Feature engineering for anomaly detection: expenditure

In [7]:
# Calculating Q1, Q3, and IQR for each account in 'from_totally_fake_account'
quartiles = expenditures_training_df.groupby('Account No')['Amount'].quantile([0.25, 0.5, 0.75]).unstack()
quartiles['IQR'] = quartiles[0.75] - quartiles[0.25]

thresholds = quartiles.copy()
thresholds['Overall_Threshold_1_IQR'] = quartiles[0.75] + 1.5 * quartiles['IQR']
thresholds['Overall_Threshold_2_IQR'] = quartiles[0.75] + 3 * quartiles['IQR']
thresholds['Overall_Threshold_3_IQR'] = quartiles[0.75] + 5 * quartiles['IQR']

anomaly_detection_by_expenditures_df = thresholds


anomaly_detection_by_expenditures_df

,0.25,0.5,0.75,IQR,Overall_Threshold_1_IQR,Overall_Threshold_2_IQR,Overall_Threshold_3_IQR
Account No,,,,,,,
101531259.0,25.2200,78.000,120.8750,95.6550,264.35750,407.8400,599.1500
104832000.0,16.5125,49.785,106.7800,90.2675,242.18125,377.5825,558.1175
105375973.0,18.9900,43.725,102.3650,83.3750,227.42750,352.4900,519.2400
106601471.0,10.6900,18.705,45.5975,34.9075,97.95875,150.3200,220.1350
108481285.0,10.3350,21.980,45.6275,35.2925,98.56625,151.5050,222.0900
...,...,...,...,...,...,...,...
995615876.0,25.8750,53.850,108.9750,83.1000,233.62500,358.2750,524.4750
996042490.0,15.9900,34.730,93.4750,77.4850,209.70250,325.9300,480.9000
998390769.0,15.5600,74.790,145.4825,129.9225,340.36625,535.2500,795.0950


### Creating thresholds for each user to user transfers

In [19]:
def calculate_thresholds(group, multiplier):
    quantiles = group['Amount'].quantile([0.25, 0.75])
    IQR = quantiles[0.75] - quantiles[0.25]
    threshold = quantiles[0.75] + multiplier * IQR
    return threshold

# Applying the adjusted function for each threshold calculation with different multipliers

# For Threshold 1 (1.5 * IQR)
thresholds_1 = expenditures_training_df.dropna(subset=['Account No', 'Third Party Account No']).groupby(['Account No', 'Third Party Account No']).apply(lambda group: calculate_thresholds(group, 1.5)).reset_index(name='Threshold_1')
thresholds_structured_1 = thresholds_1.groupby('Account No').apply(lambda x: dict(zip(x['Third Party Account No'], x['Threshold_1']))).to_dict()

# For Threshold 2 (3 * IQR)
thresholds_2 = expenditures_training_df.dropna(subset=['Account No', 'Third Party Account No']).groupby(['Account No', 'Third Party Account No']).apply(lambda group: calculate_thresholds(group, 3)).reset_index(name='Threshold_2')
thresholds_structured_2 = thresholds_2.groupby('Account No').apply(lambda x: dict(zip(x['Third Party Account No'], x['Threshold_2']))).to_dict()

# For Threshold 3 (5 * IQR)
thresholds_3 = expenditures_training_df.dropna(subset=['Account No', 'Third Party Account No']).groupby(['Account No', 'Third Party Account No']).apply(lambda group: calculate_thresholds(group, 5)).reset_index(name='Threshold_3')
thresholds_structured_3 = thresholds_3.groupby('Account No').apply(lambda x: dict(zip(x['Third Party Account No'], x['Threshold_3']))).to_dict()

anomaly_detection_by_expenditures_df['Third_Party_Account_Threshold_1'] = anomaly_detection_by_expenditures_df.index.map(lambda x: thresholds_structured_1.get(x, np.nan))
anomaly_detection_by_expenditures_df['Third_Party_Account_Threshold_2'] = anomaly_detection_by_expenditures_df.index.map(lambda x: thresholds_structured_2.get(x, np.nan))
anomaly_detection_by_expenditures_df['Third_Party_Account_Threshold_3'] = anomaly_detection_by_expenditures_df.index.map(lambda x: thresholds_structured_3.get(x, np.nan))

In [18]:
anomaly_detection_by_expenditures_df

,0.25,0.5,0.75,IQR,Overall_Threshold_1_IQR,Overall_Threshold_2_IQR,Overall_Threshold_3_IQR,Third_Party_Account_Threshold_1,Third_Party_Account_Threshold_2,Third_Party_Account_Threshold_3
Account No,,,,,,,,,,
101531259.0,25.2200,78.000,120.8750,95.6550,264.35750,407.8400,599.1500,NaN,NaN,NaN
104832000.0,16.5125,49.785,106.7800,90.2675,242.18125,377.5825,558.1175,{587923845.0: 670.0},{587923845.0: 670.0},{587923845.0: 670.0}
105375973.0,18.9900,43.725,102.3650,83.3750,227.42750,352.4900,519.2400,NaN,NaN,NaN
106601471.0,10.6900,18.705,45.5975,34.9075,97.95875,150.3200,220.1350,NaN,NaN,NaN
108481285.0,10.3350,21.980,45.6275,35.2925,98.56625,151.5050,222.0900,{913801979.0: 2307.5},{913801979.0: 2797.5},{913801979.0: 3287.5}
...,...,...,...,...,...,...,...,...,...,...
995615876.0,25.8750,53.850,108.9750,83.1000,233.62500,358.2750,524.4750,NaN,NaN,NaN
996042490.0,15.9900,34.730,93.4750,77.4850,209.70250,325.9300,480.9000,{792845450.0: 1500.0},{792845450.0: 1500.0},{792845450.0: 1500.0}
998390769.0,15.5600,74.790,145.4825,129.9225,340.36625,535.2500,795.0950,NaN,NaN,NaN


### Creating thresholds for each user to industry transactions

In [21]:
industry_categories_expenditure = {
    "Coffee": ["Coffee #1", "Costa Coffee", "Starbucks"],
    "Technology and Gaming": ["Mojang Studios", "Xbox", "Blizzard", "SquareOnix"],
    "Entertainment and Media": ["Netflix", "Disney", "Amazon"],
    "Retail - Crafts, Hobbies and Books": ["Wool", "Etsy", "Hobby Lobby", "Cass Art", "Craftastic", "Hobbycraft", "The Works", "Blackwell's", "Brilliant Brushes", "Five Senses Art", "A Yarn Story", "Foyles", "AMAZON", "Stitch By Stitch", "Fitted Stitch"],
    "Retail - Clothing and Fashion": ["Reebok", "Sports Direct", "JD Sports", "Tesco", "Matalan", "Fat Face", "Topshop", "Revella", "Mountain Warehouse", "Millets", "Loosely Fitted"],
    "Retail - General and Department Stores": ["Coop Local", "Sainsbury Local", "Sainsbury", "Selfridges"],
    "Retail - High end": ["Barbiee Boutique", "North Face", "Collector Cave"],
    "Retail- Kids": ["Mamas & Papas", "Mothercare", "Gap Kids"],
    "Retail - services ": ["Vision Express", "Pets Corner", "Head", "A Cut Above", "Specsavers"],
    "Health and Fitness": ["PureGym", "Grand Union BJJ"],
    "Pharmacy": ["Boots", "Lloyds Pharmacy"],
    "Hospital and Care Home": ["University College Hospital", "Happy Days Home", "Remedy plus care"],
    "Electronics and Entertainment Retail": ["CeX", "Gamestation", "HMV"],
    "Child Care and Education": ["Lavender Primary", "Town High", "RugbyFields"],
    "Care Services": ["Happy Days Home", "Remedy plus care", "Kew House"],
    "Financial Services": ["LBG", "Halifax"],
    "Pubs": ["Kings Arms", "The Crown", "Rose & Crown"],
    "Takeaway": ["Deliveroo", "JustEat"],
    "Restaurant": ["Frankie & Bennies"],
    "Park": ["Victoria Park"]
}

In [30]:
# Add industry column to tx_df

def tx_df_industry_mapping(name):
    if isinstance(name, str):  # Check if input is a string
        for category, names_list in industry_categories_expenditure.items():
            if any(n in name for n in names_list):
                return category
    return np.nan


# Apply the mapping
expenditures_training_df['industry_category'] = expenditures_training_df['Third Party Name'].apply(tx_df_industry_mapping)
expenditures_training_df

/var/folders/rw/215myrns4md_n1k8nkyww6mw0000gn/T/ipykernel_32303/1366314935.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  expenditures_training_df['industry_category'] = expenditures_training_df['Third Party Name'].apply(tx_df_industry_mapping)


,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name,Datetime,industry_category
108236,21/06/2023,09:03:00,809499628.0,10775.476914,48.10,NaN,Tesco,2023-06-21 09:03:00,Retail - Clothing and Fashion
204684,02/11/2023,19:22:00,518474507.0,15455.325500,29.19,NaN,Deliveroo,2023-11-02 19:22:00,Takeaway
19823,30/01/2023,11:29:00,655368335.0,2230.145000,99.98,NaN,Gamestation,2023-01-30 11:29:00,Electronics and Entertainment Retail
151016,25/08/2023,11:09:00,678851253.0,4313.628611,63.70,NaN,Fat Face,2023-08-25 11:09:00,Retail - Clothing and Fashion
26217,31/01/2023,23:59:00,163470640.0,60.014100,12.99,NaN,SquareOnix,2023-01-31 23:59:00,Technology and Gaming
...,...,...,...,...,...,...,...,...,...
119879,03/07/2023,18:54:00,424397946.0,2450.930784,119.62,NaN,Sainsbury,2023-07-03 18:54:00,Retail - General and Department Stores
103694,11/06/2023,10:27:00,241459011.0,995.080013,15.98,NaN,Sports Direct,2023-06-11 10:27:00,Retail - Clothing and Fashion
131932,29/07/2023,14:02:00,744211756.0,-3127.708250,44.72,NaN,Tesco,2023-07-29 14:02:00,Retail - Clothing and Fashion
146867,16/08/2023,17:57:00,245690309.0,7640.773332,46.45,NaN,Tesco,2023-08-16 17:57:00,Retail - Clothing and Fashion


In [32]:
# Re-define the calculate_thresholds function in case it's not defined
def calculate_thresholds(group, multiplier):
    quantiles = group['Amount'].quantile([0.25, 0.75])
    IQR = quantiles[0.75] - quantiles[0.25]
    threshold = quantiles[0.75] + multiplier * IQR
    return threshold

# Assuming business_accounts_df is already defined
grouped_by_industry = expenditures_training_df.groupby(['Account No', 'industry_category'])

# Calculate thresholds for each grouping
thresholds_structured_by_industry_and_category = {}
for (acc, cat), group in grouped_by_industry:
    thresholds_structured_by_industry_and_category[(acc, cat)] = {
        'threshold_1_IQR': calculate_thresholds(group, 1.5),
        'threshold_2_IQR': calculate_thresholds(group, 3),
        'threshold_3_IQR': calculate_thresholds(group, 5)
    }

# Create a mapping function that retrieves threshold values for a given user and industry
def map_thresholds_to_user(user_id, threshold_key):
    return {cat: details[threshold_key] for (acc, cat), details in thresholds_structured_by_industry_and_category.items() if acc == user_id}

# Convert the index to a regular column
anomaly_detection_by_expenditures_df.reset_index(inplace=True)

# Apply the mapping function for each threshold level
anomaly_detection_by_expenditures_df['industry_threshold_1_IQR'] = anomaly_detection_by_expenditures_df['Account No'].apply(lambda x: map_thresholds_to_user(x, 'threshold_1_IQR'))
anomaly_detection_by_expenditures_df['industry_threshold_2_IQR'] = anomaly_detection_by_expenditures_df['Account No'].apply(lambda x: map_thresholds_to_user(x, 'threshold_2_IQR'))
anomaly_detection_by_expenditures_df['industry_threshold_3_IQR'] = anomaly_detection_by_expenditures_df['Account No'].apply(lambda x: map_thresholds_to_user(x, 'threshold_3_IQR'))
anomaly_detection_by_expenditures_df

,Account No,0.25,0.5,0.75,IQR,Overall_Threshold_1_IQR,Overall_Threshold_2_IQR,Overall_Threshold_3_IQR,Third_Party_Account_Threshold_1,Third_Party_Account_Threshold_2,Third_Party_Account_Threshold_3,industry_threshold_1_IQR,industry_threshold_2_IQR,industry_threshold_3_IQR
0,101531259.0,25.2200,78.000,120.8750,95.6550,264.35750,407.8400,599.1500,NaN,NaN,NaN,"{'Entertainment and Media': 7.99, 'Financial S...","{'Entertainment and Media': 7.99, 'Financial S...","{'Entertainment and Media': 7.99, 'Financial S..."
1,104832000.0,16.5125,49.785,106.7800,90.2675,242.18125,377.5825,558.1175,{587923845.0: 670.0},{587923845.0: 670.0},{587923845.0: 670.0},{'Electronics and Entertainment Retail': 189.9...,{'Electronics and Entertainment Retail': 272.4...,{'Electronics and Entertainment Retail': 382.3...
2,105375973.0,18.9900,43.725,102.3650,83.3750,227.42750,352.4900,519.2400,NaN,NaN,NaN,{'Entertainment and Media': 27.990000000000002...,"{'Entertainment and Media': 39.99, 'Health and...","{'Entertainment and Media': 55.99, 'Health and..."
3,106601471.0,10.6900,18.705,45.5975,34.9075,97.95875,150.3200,220.1350,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 178.7...,{'Electronics and Entertainment Retail': 234.9...,{'Electronics and Entertainment Retail': 309.9...
4,108481285.0,10.3350,21.980,45.6275,35.2925,98.56625,151.5050,222.0900,{913801979.0: 2185.0},{913801979.0: 2552.5},{913801979.0: 3042.5},"{'Entertainment and Media': 29.73375, 'Health ...","{'Entertainment and Media': 42.78, 'Health and...","{'Entertainment and Media': 60.175, 'Health an..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,995615876.0,25.8750,53.850,108.9750,83.1000,233.62500,358.2750,524.4750,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 126.2...,{'Electronics and Entertainment Retail': 174.9...,{'Electronics and Entertainment Retail': 239.9...
972,996042490.0,15.9900,34.730,93.4750,77.4850,209.70250,325.9300,480.9000,{792845450.0: 1500.0},{792845450.0: 1500.0},{792845450.0: 1500.0},{'Electronics and Entertainment Retail': 237.4...,{'Electronics and Entertainment Retail': 349.9...,{'Electronics and Entertainment Retail': 499.8...
973,998390769.0,15.5600,74.790,145.4825,129.9225,340.36625,535.2500,795.0950,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 219.9...,{'Electronics and Entertainment Retail': 309.9...,{'Electronics and Entertainment Retail': 429.9...
974,998405607.0,15.9900,35.000,100.0000,84.0100,226.01500,352.0300,520.0500,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 154.9...,{'Electronics and Entertainment Retail': 192.4...,{'Electronics and Entertainment Retail': 242.4...


In [35]:
# Extracting keys from 'transfer_to_accounts_threshold_1_IQR' to create 'user_user_interaction'
anomaly_detection_by_expenditures_df['user_user_interaction'] = anomaly_detection_by_expenditures_df['Third_Party_Account_Threshold_1'].apply(lambda x: list(x.keys()) if isinstance(x, dict) else None)

# Extracting keys from 'industry_threshold_1_IQR' to create 'user_industry_interaction'
anomaly_detection_by_expenditures_df['user_industry_interaction'] = anomaly_detection_by_expenditures_df['industry_threshold_1_IQR'].apply(lambda x: list(x.keys()) if isinstance(x, dict) else None)

anomaly_detection_by_expenditures_df


,Account No,0.25,0.5,0.75,IQR,Overall_Threshold_1_IQR,Overall_Threshold_2_IQR,Overall_Threshold_3_IQR,Third_Party_Account_Threshold_1,Third_Party_Account_Threshold_2,Third_Party_Account_Threshold_3,industry_threshold_1_IQR,industry_threshold_2_IQR,industry_threshold_3_IQR,user_user_interaction,user_industry_interaction
0,101531259.0,25.2200,78.000,120.8750,95.6550,264.35750,407.8400,599.1500,NaN,NaN,NaN,"{'Entertainment and Media': 7.99, 'Financial S...","{'Entertainment and Media': 7.99, 'Financial S...","{'Entertainment and Media': 7.99, 'Financial S...",None,"[Entertainment and Media, Financial Services, ..."
1,104832000.0,16.5125,49.785,106.7800,90.2675,242.18125,377.5825,558.1175,{587923845.0: 670.0},{587923845.0: 670.0},{587923845.0: 670.0},{'Electronics and Entertainment Retail': 189.9...,{'Electronics and Entertainment Retail': 272.4...,{'Electronics and Entertainment Retail': 382.3...,[587923845.0],"[Electronics and Entertainment Retail, Enterta..."
2,105375973.0,18.9900,43.725,102.3650,83.3750,227.42750,352.4900,519.2400,NaN,NaN,NaN,{'Entertainment and Media': 27.990000000000002...,"{'Entertainment and Media': 39.99, 'Health and...","{'Entertainment and Media': 55.99, 'Health and...",None,"[Entertainment and Media, Health and Fitness, ..."
3,106601471.0,10.6900,18.705,45.5975,34.9075,97.95875,150.3200,220.1350,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 178.7...,{'Electronics and Entertainment Retail': 234.9...,{'Electronics and Entertainment Retail': 309.9...,None,"[Electronics and Entertainment Retail, Enterta..."
4,108481285.0,10.3350,21.980,45.6275,35.2925,98.56625,151.5050,222.0900,{913801979.0: 2185.0},{913801979.0: 2552.5},{913801979.0: 3042.5},"{'Entertainment and Media': 29.73375, 'Health ...","{'Entertainment and Media': 42.78, 'Health and...","{'Entertainment and Media': 60.175, 'Health an...",[913801979.0],"[Entertainment and Media, Health and Fitness, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,995615876.0,25.8750,53.850,108.9750,83.1000,233.62500,358.2750,524.4750,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 126.2...,{'Electronics and Entertainment Retail': 174.9...,{'Electronics and Entertainment Retail': 239.9...,None,"[Electronics and Entertainment Retail, Financi..."
972,996042490.0,15.9900,34.730,93.4750,77.4850,209.70250,325.9300,480.9000,{792845450.0: 1500.0},{792845450.0: 1500.0},{792845450.0: 1500.0},{'Electronics and Entertainment Retail': 237.4...,{'Electronics and Entertainment Retail': 349.9...,{'Electronics and Entertainment Retail': 499.8...,[792845450.0],"[Electronics and Entertainment Retail, Enterta..."
973,998390769.0,15.5600,74.790,145.4825,129.9225,340.36625,535.2500,795.0950,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 219.9...,{'Electronics and Entertainment Retail': 309.9...,{'Electronics and Entertainment Retail': 429.9...,None,"[Electronics and Entertainment Retail, Enterta..."
974,998405607.0,15.9900,35.000,100.0000,84.0100,226.01500,352.0300,520.0500,NaN,NaN,NaN,{'Electronics and Entertainment Retail': 154.9...,{'Electronics and Entertainment Retail': 192.4...,{'Electronics and Entertainment Retail': 242.4...,None,"[Electronics and Entertainment Retail, Enterta..."


### Trialling with standard deviations

In [ ]:
# Calculate the mean and std for each account
mean_std = expenditures_training_df.groupby('Account No')['Amount'].agg(['mean', 'std'])

# Establishing thresholds based on standard deviations from the mean
mean_std['Threshold_1_STD'] = mean_std['mean'] + mean_std['std'] * 1  # 1 std from the mean
mean_std['Threshold_2_STD'] = mean_std['mean'] + mean_std['std'] * 2  # 2 stds from the mean
mean_std['Threshold_3_STD'] = mean_std['mean'] + mean_std['std'] * 3  # 3 stds from the mean

# Your anomaly detection DataFrame now is mean_std with the standard deviation thresholds
anomaly_detection_by_expenditures_std_df = mean_std

# Renaming the index to 'Account No' if needed
anomaly_detection_by_expenditures_std_df.index.rename('Account No', inplace=True)

anomaly_detection_by_expenditures_std_df


### Feature engineering for anomaly detection: payments

In [20]:
# Calculating Q1, Q3, and IQR for each account in 'from_totally_fake_account'
quartiles = payments_training_df.groupby('Account No')['Amount'].quantile([0.25, 0.5, 0.75]).unstack()
quartiles['IQR'] = quartiles[0.75] - quartiles[0.25]

thresholds = quartiles.copy()
thresholds['Overall_Threshold_1_IQR'] = quartiles[0.75] + 1.5 * quartiles['IQR']
thresholds['Overall_Threshold_2_IQR'] = quartiles[0.75] + 3 * quartiles['IQR']
thresholds['Overall_Threshold_3_IQR'] = quartiles[0.75] + 5 * quartiles['IQR']

anomaly_detection_by_payments_df = thresholds

anomaly_detection_by_payments_df.index.rename('Account No', inplace=True)

anomaly_detection_by_payments_df

,0.25,0.5,0.75,IQR,Overall_Threshold_1_IQR,Overall_Threshold_2_IQR,Overall_Threshold_3_IQR
Account No,,,,,,,
101531259.0,1542.5,1542.5,1542.5,0.0,1542.5,1542.5,1542.5
104832000.0,1675.0,1675.0,1675.0,0.0,1675.0,1675.0,1675.0
105375973.0,900.0,900.0,900.0,0.0,900.0,900.0,900.0
106601471.0,1675.0,1675.0,1675.0,0.0,1675.0,1675.0,1675.0
108481285.0,640.0,640.0,1584.0,944.0,3000.0,4416.0,6304.0
...,...,...,...,...,...,...,...
995615876.0,1733.0,1733.0,1733.0,0.0,1733.0,1733.0,1733.0
996042490.0,3750.0,3750.0,3750.0,0.0,3750.0,3750.0,3750.0
998390769.0,1625.0,1625.0,1625.0,0.0,1625.0,1625.0,1625.0
